# Defining Tools

A `Tool` consists of:
1. A name that is used to uniquely identify the tool so the LM can specify which tool it intends to call
2. A description that is used to help indicate to the LM when it is appropriate to call the tool
3. A callable that is executed when the tool is actually called
4. A constraint that ensures that the LM's *output* is a valid *input* to the callable

## Functions
The most common type of tool is defined by a function.

The constraint on the LM's output can be specified by a JSON schema, where the JSON schema should have type `object` and have `properties` matching the names of the function's arguments.

Note: positional-only and variadic arguments are not yet supported.

In [1]:
import random
from typing import Literal

def get_weather_func(city: str, unit: Literal["celsius", "fahrenheit"] = "celsius") -> str:
    """
    Get the current weather for a given city.
    """
    temp = random.randint(-10, 35)
    if temp <= 0:
        weather = "snowy"
    elif temp <= 20:
        weather = "cloudy"
    else:
        weather = "sunny"
    if unit == "fahrenheit":
        temp = temp * 9/5 + 32
    return f"The current temperature in {city} is {temp} degrees {unit} and it is {weather}."

`Tool.from_callable` takes a function and creates a `Tool` object.

- The name of the tool will be inferred from the function's name
- The description of the tool will be inferred from the function's docstring
- The JSON schema of the parameters will be inferred from any type annotations

In [2]:
from guidance.types import Tool

Tool.from_callable(
    callable=get_weather_func
).model_dump()

{'name': 'get_weather_func',
 'description': 'Get the current weather for a given city.',
 'tool': {'type': 'function',
  'parameters': {'additionalProperties': False,
   'properties': {'city': {'title': 'City', 'type': 'string'},
    'unit': {'enum': ['celsius', 'fahrenheit'],
     'title': 'Unit',
     'type': 'string'}},
   'required': ['city', 'unit'],
   'title': 'get_weather_func',
   'type': 'object'}},
 'callable': <function __main__.get_weather_func(city: str, unit: Literal['celsius', 'fahrenheit'] = 'celsius') -> str>}

The name, description, and parameters (may be a JSON schema or a pydantic.BaseModel class) can optionally be specified directly: 

In [3]:
from pydantic import BaseModel, Field, ConfigDict

class GetWeatherArgs(BaseModel):
    city: str = Field(..., description="The name of the city to get the weather for.")
    unit: Literal["celsius", "fahrenheit"] = Field("celsius", description="The unit of temperature to return.")

    model_config = ConfigDict(
        extra="forbid", # Disallow extra fields -- required for OpenAI
    )

get_weather_tool = Tool.from_callable(
    callable=get_weather_func,
    name="get_weather",
    description="Get the current weather for a given city.",
    parameters=GetWeatherArgs
)
get_weather_tool.model_dump()

{'name': 'get_weather',
 'description': 'Get the current weather for a given city.',
 'tool': {'type': 'function',
  'parameters': {'additionalProperties': False,
   'properties': {'city': {'description': 'The name of the city to get the weather for.',
     'title': 'City',
     'type': 'string'},
    'unit': {'default': 'celsius',
     'description': 'The unit of temperature to return.',
     'enum': ['celsius', 'fahrenheit'],
     'title': 'Unit',
     'type': 'string'}},
   'required': ['city'],
   'title': 'GetWeatherArgs',
   'type': 'object'}},
 'callable': <function __main__.get_weather_func(city: str, unit: Literal['celsius', 'fahrenheit'] = 'celsius') -> str>}

## Custom Tools

### Regex tools
We can use a regex to define the type of structured input that our tool expects. In this example, our tool is a `whois` lookup that expects a well-formatted url.

Note that unlike `Tool.from_callable`, the callable passed to `Tool.from_regex` must take exactly one argument, which will be passed as a *string matching the given regex*.

In [4]:
import subprocess
from urllib.parse import urlparse

def whois_lookup(url: str) -> str:
    domain = urlparse(url).netloc
    result = subprocess.run(["whois", domain], capture_output=True, text=True, timeout=30)
    return result.stdout

whois_tool = Tool.from_regex(
    pattern=r"https?:\/\/[^\s]+",
    name="whois_lookup",
    description="A tool that performs a WHOIS lookup on a domain.",
    callable=whois_lookup
)
whois_tool.model_dump()

{'name': 'whois_lookup',
 'description': 'A tool that performs a WHOIS lookup on a domain.',
 'tool': {'type': 'custom',
  'format': {'type': 'grammar',
   'syntax': 'regex',
   'definition': 'https?:\\/\\/[^\\s]+'}},
 'callable': <function __main__.whois_lookup(url: str) -> str>}

### Guidance grammar tools
In addition to regex tools, we can define guidance grammars that constrain the model's output to match our tool's expected input format.

Below, we define a grammar that describes simple arithmetic expressions, and our tool will evaluate these expressions as python numbers.

Again, note that the callable passed to `Tool.from_grammar` must take exactly one argument, which will be passed as a *string matching the given grammar*.

In [5]:
from guidance import guidance, select, regex

@guidance(stateless=True)
def expression(lm):
    return lm + select([
        term(),
        expression() + "+" + term(),
        expression() + "-" + term(),
    ])

@guidance(stateless=True)
def term(lm):
    return lm + select([
        factor(),
        term() + "*" + factor(),
        term() + "/" + factor(),
    ])

@guidance(stateless=True)
def factor(lm):
    return lm + select([number(), "(" + expression() + ")"])

@guidance(stateless=True)
def number(lm):
    return lm + regex(r'\d+(\.\d+)?')

In [6]:
from sympy import sympify
from ast import literal_eval
from typing import Union

def evaluate_expression(expr: str) -> Union[int, float]:
    sympy_expr = sympify(expr)
    if not sympy_expr.is_number:
        raise ValueError(f"Invalid expression: {expr}")
    return literal_eval(str(sympy_expr))

calculator = Tool.from_grammar(
    grammar=expression(),
    name="calculator",
    description="A calculator that can evaluate mathematical expressions.",
    callable=evaluate_expression
)
calculator.model_dump()

{'name': 'calculator',
 'description': 'A calculator that can evaluate mathematical expressions.',
 'tool': {'type': 'custom',
  'format': {'type': 'grammar',
   'syntax': 'lark',
   'definition': '%llguidance {}\n\nstart: expression\n\nexpression: term\n     | expression "+" term\n     | expression "-" term\n\nterm: factor\n     | term "*" factor\n     | term "/" factor\n\nfactor: NUMBER\n     | "(" expression ")"\n\nNUMBER: /\\d+(\\.\\d+)?/\n'}},
 'callable': <function __main__.evaluate_expression(expr: str) -> Union[int, float]>}

### Lark tools
Under the hood, guidance grammars just compile down to a [lark-like syntax for describing context free grammars](https://github.com/guidance-ai/llguidance/blob/main/docs/syntax.md).

We can create a tool using this lark syntax directly. Note that the same caveats about arguments to the callable still apply.


In [7]:
lark = r"""
start: expression

expression: term
     | expression "+" term
     | expression "-" term

term: factor
     | term "*" factor
     | term "/" factor

factor: NUMBER
     | "(" expression ")"

NUMBER: /\d+(\.\d+)?/
"""

calculator_tool = Tool.from_lark(
    lark=lark,
    name="calculator",
    description="A calculator that can evaluate mathematical expressions.",
    callable=evaluate_expression
)
calculator_tool.model_dump()

{'name': 'calculator',
 'description': 'A calculator that can evaluate mathematical expressions.',
 'tool': {'type': 'custom',
  'format': {'type': 'grammar',
   'syntax': 'lark',
   'definition': '\nstart: expression\n\nexpression: term\n     | expression "+" term\n     | expression "-" term\n\nterm: factor\n     | term "*" factor\n     | term "/" factor\n\nfactor: NUMBER\n     | "(" expression ")"\n\nNUMBER: /\\d+(\\.\\d+)?/\n'}},
 'callable': <function __main__.evaluate_expression(expr: str) -> Union[int, float]>}

# Calling Tools

In [8]:
tools = [
    get_weather_tool,
    whois_tool,
    calculator_tool,
]

Most remote models have native support for tool-calling with function (JSON-specified) tools.

With OpenAI's release of GPT-5, [they additionally support tools that are specified by regex or lark-style context free grammars](https://platform.openai.com/docs/guides/function-calling#custom-tools). In these cases, model sampling is constrained using [LLGuidance](https://github.com/guidance-ai/llguidance/blob/main/docs/syntax.md)

To allow our LM to use our tools, we simply pass them in a list as an argument to `gen`. If the LM wants to call one of our tools, `guidance` will handle the callback for us:

In [14]:
from guidance.models import OpenAI
from guidance import user, assistant, gen
lm = OpenAI("gpt-5-mini")

with user():
    lm += "What is the weather like in San Francisco?"
with assistant():
    lm += gen(tools=tools, tool_choice="required")
with assistant():
    lm += gen()

StitchWidget(initial_height='auto', initial_width='100%', srcdoc='<!doctype html>\n<html lang="en">\n<head>\n …

You may notice that the tool call is wrapped in `<function={name}>...</function>` tags and the result is wrapped in `<function_result>..</function_result>` tags. This is purely for visualization purposes, as OpenAI uses a structured representation under the hood which we (somewhat arbitrarily) represent as a string above.

In [15]:
lm = OpenAI("gpt-5-mini")

with user():
    lm += "If Johnny has 5 apples and gives 2 to Mary, how many apples does he have left?"
with assistant():
    lm += gen(tools=tools, tool_choice="required")
with assistant():
    lm += gen()

StitchWidget(initial_height='auto', initial_width='100%', srcdoc='<!doctype html>\n<html lang="en">\n<head>\n …

In [16]:
lm = OpenAI("gpt-5-mini")

with user():
    lm += "Who owns the openai website?"
with assistant():
    lm += gen(tools=tools, tool_choice="required")
with assistant():
    lm += gen()

StitchWidget(initial_height='auto', initial_width='100%', srcdoc='<!doctype html>\n<html lang="en">\n<head>\n …